In [20]:
# Test interaction with sample dataset

import os
import pandas as pd

IN_FILE = os.environ.get("IN_FILE", "../lambda/data/sample-file-assessment.snappy.parquet")

df = pd.read_parquet(IN_FILE)
df.head()

,name,value,start_date,end_date,year_week,has_subtrackers,token,dataplatform_inserted_at,country,os_name
0,Website,0.0,2020-12-17,2020-12-17,2020_51,True,17xptrn,2020-12-24 13:12:53.538505,GB,ios
1,Email,0.0,2020-12-17,2020-12-17,2020_51,True,dy3vti6,2020-12-24 13:12:53.538505,GB,ios
2,Instagram Installs,1.0,2020-12-17,2020-12-17,2020_51,True,ew373nn,2020-12-24 13:12:53.538505,GB,ios
3,Google Ads UAC,7.0,2020-12-17,2020-12-17,2020_51,True,f91turk,2020-12-24 13:12:53.538505,GB,ios
4,Facebook Installs,1.0,2020-12-17,2020-12-17,2020_51,True,iongxw7,2020-12-24 13:12:53.538505,GB,ios


In [21]:
# Check the datatypes for each column
df.dtypes

name                                object
value                              float32
start_date                  datetime64[ns]
end_date                    datetime64[ns]
year_week                           object
has_subtrackers                    boolean
token                               object
dataplatform_inserted_at    datetime64[ns]
country                             object
os_name                             object
dtype: object

In [22]:
# Check the count of unique values in name. Indicates we can use category not string types here
df["name"].value_counts()

ALL    5282
GB     2422
IT      950
FR      676
Name: country, dtype: int64

In [23]:
old_mem_usage = df["name"].nbytes
print(f"Name column currently using {old_mem_usage} bytes")

Name column currently using 74640 bytes


In [24]:
# Convert name to a categorical datatype
df["name"] = df["name"].astype("category")
# Value can be changed to int
df["value"] = df["value"].astype("int")

In [25]:
# Mem reduction by type change
print(f"Now {df['name'].nbytes} : {old_mem_usage - df['name'].nbytes} bytes reduced by switching to category dtype")

Now 9538 : 65102 bytes reduced by switching to category dtype


In [26]:
# Country and os_name can also be categories
df[["country","os_name","year_week"]] = df[["country","os_name","year_week"]].astype("category")

In [27]:
# Check if we have any empty values in our dateset ~ would indicate potential for cleaning
df.isnull().values.any()

False

In [28]:
# Potenital column duplication if start_date always == end_date.
len(df[df["start_date"] != df["end_date"]]) == 0

True

In [29]:
# Example Transform: Computing the weekly highest values for each name

# Drop columns we will not use - everything after year_week and start+end date columns
df.drop(columns=list(df.columns[5:])+["start_date","end_date"],inplace=True)
df.head()

,name,value,year_week
0,Website,0,2020_51
1,Email,0,2020_51
2,Instagram Installs,1,2020_51
3,Google Ads UAC,7,2020_51
4,Facebook Installs,1,2020_51


In [30]:
# Sum the values by name category for each year_week
df["week_total"] = df.groupby(['year_week','name'])["value"].transform('sum')
df.head()

,name,value,year_week,week_total
0,Website,0,2020_51,18
1,Email,0,2020_51,0
2,Instagram Installs,1,2020_51,180
3,Google Ads UAC,7,2020_51,1090
4,Facebook Installs,1,2020_51,80


In [31]:
# Compute the max value for each week
df["week_max"] = df.groupby(['year_week'])["week_total"].transform('max')
df.head()

,name,value,year_week,week_total,week_max
0,Website,0,2020_51,18,2536
1,Email,0,2020_51,0,2536
2,Instagram Installs,1,2020_51,180,2536
3,Google Ads UAC,7,2020_51,1090,2536
4,Facebook Installs,1,2020_51,80,2536


In [32]:
# Select the rows where the Weekly Total is the Weekly Max
df = df[df["week_max"] == df["week_total"]]
df.head()

,name,value,year_week,week_total,week_max
9,Organic,46,2020_51,2536,2536
23,Organic,33,2020_51,2536,2536
35,Organic,79,2020_51,2536,2536
44,Organic,4,2020_51,2536,2536
46,Organic,0,2020_51,2536,2536


In [33]:
# Remove the duplicate (name,year_week) keys and reset the indexes
df.drop_duplicates(["name","year_week"], inplace=True)


df.reset_index(drop=True, inplace=True)
df.head()

,name,value,year_week,week_total,week_max
0,Organic,46,2020_51,2536,2536
1,Organic,3,2020_40,3362,3362
2,Organic,3,2020_41,5468,5468
3,Organic,2,2020_42,5552,5552
4,Organic,1,2020_43,5998,5998


In [34]:
df.drop(columns=["value","week_max"],inplace=True, errors="ignore")
df.head()

,name,year_week,week_total
0,Organic,2020_51,2536
1,Organic,2020_40,3362
2,Organic,2020_41,5468
3,Organic,2020_42,5552
4,Organic,2020_43,5998


In [35]:
# Use the year_week as index and sort
df.set_index('year_week',inplace=True)
df.sort_index(inplace=True)
df.rename(columns = {'week_total':'count'}, inplace = True)
df.head()

,name,count
year_week,,
2020_40,Organic,3362
2020_41,Organic,5468
2020_42,Organic,5552
2020_43,Organic,5998
2020_44,Organic,6148


In [36]:
# Dataframe now shows the count of the most popular name indexed by each week
# Saved to S3 in lambda function